In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Database configuration — CHANGE these as per your setup
db_config = {
    'host': 'localhost',
    'user': input("Enter MySQL username: "),
    'password': input("Enter MySQL password: "),
    'database': 'supermarket_db'
}


In [2]:
try:
    conn = mysql.connector.connect(**db_config)
    if conn.is_connected():
        print("Successfully connected to MySQL database.")
        cursor = conn.cursor()
except Error as e:
    print(f"Error connecting to MySQL: {e}")


Successfully connected to MySQL database.


In [3]:
print("\n--- Fetching and Preprocessing Data ---")

sql_query = """
SELECT invoice_id, city, gender, product_line, unit_price, quantity, total, customer_type FROM sales
"""

# Load data from MySQL into pandas DataFrame
df = pd.read_sql(sql_query, conn)
print(f"Loaded {len(df)} records.")

# One-hot encoding categorical variables
df_processed = pd.get_dummies(df, columns=['city', 'gender', 'product_line'], drop_first=True)

# Label encode target variable
le = LabelEncoder()
df_processed['customer_type_encoded'] = le.fit_transform(df_processed['customer_type'])
print(f"Target variable 'customer_type' encoded. Mapping: {list(zip(le.classes_, le.transform(le.classes_)))}")



--- Fetching and Preprocessing Data ---
Loaded 1000 records.
Target variable 'customer_type' encoded. Mapping: [('Member', np.int64(0)), ('Normal', np.int64(1))]


/tmp/ipykernel_339503/2436555951.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [4]:
print("\n--- Preparing data for training ---")

features = [col for col in df_processed.columns if col not in ['invoice_id', 'customer_type', 'customer_type_encoded']]
X = df_processed[features]
y = df_processed['customer_type_encoded']

# Split data 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")



--- Preparing data for training ---
Train set size: 800 samples
Test set size: 200 samples


In [5]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print("Model training complete")

Model training complete


In [6]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy on Test Data: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))



Model Accuracy on Test Data: 0.52

Classification Report:
              precision    recall  f1-score   support

      Member       0.53      0.46      0.49       102
      Normal       0.50      0.57      0.54        98

    accuracy                           0.52       200
   macro avg       0.52      0.52      0.51       200
weighted avg       0.52      0.52      0.51       200



In [7]:
print("\n--- Simulating Predictions and Preparing for Storage ---")

# Taking first 5 rows of test set as new data
new_data_X = X_test.head(5)
original_indices = new_data_X.index
new_data_invoices = df.loc[original_indices, 'invoice_id']

# Predict on new data
new_predictions_encoded = model.predict(new_data_X)
new_predictions_labels = le.inverse_transform(new_predictions_encoded)

print("Predictions on new data:")
for inv, pred in zip(new_data_invoices, new_predictions_labels):
    print(f"  Invoice ID: {inv} -> Predicted Customer Type: {pred}")



--- Simulating Predictions and Preparing for Storage ---
Predictions on new data:
  Invoice ID: 532-59-7201 -> Predicted Customer Type: Normal
  Invoice ID: 701-69-8742 -> Predicted Customer Type: Normal
  Invoice ID: 704-10-4056 -> Predicted Customer Type: Normal
  Invoice ID: 635-28-5728 -> Predicted Customer Type: Normal
  Invoice ID: 431-66-2305 -> Predicted Customer Type: Member


In [8]:
# Create the predictions table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales_predictions (
    prediction_id INT AUTO_INCREMENT PRIMARY KEY,
    invoice_id VARCHAR(20) NOT NULL,
    predicted_customer_type VARCHAR(20),
    prediction_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (invoice_id) REFERENCES sales(invoice_id)
)
""")
print("\n'sales_predictions' table checked/created in MySQL.")

# Insert predictions into the table
insert_query = "INSERT INTO sales_predictions (invoice_id, predicted_customer_type) VALUES (%s, %s)"
predictions_to_store = list(zip(new_data_invoices, new_predictions_labels))
cursor.executemany(insert_query, predictions_to_store)
conn.commit()
print(f"{cursor.rowcount} predictions inserted into sales_predictions table.")



'sales_predictions' table checked/created in MySQL.
5 predictions inserted into sales_predictions table.


In [9]:
# Use pandas to read the data into a DataFrame for easy viewing
df_predictions = pd.read_sql("SELECT * FROM sales_predictions ORDER BY prediction_timestamp DESC", conn)

if df_predictions.empty:
    print("No prediction records found in 'sales_predictions' table.")
else:
    display(df_predictions)  # If using Jupyter notebook, display the table nicely

/tmp/ipykernel_339503/1945055905.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_predictions = pd.read_sql("SELECT * FROM sales_predictions ORDER BY prediction_timestamp DESC", conn)


,prediction_id,invoice_id,predicted_customer_type,prediction_timestamp
0,1,532-59-7201,Normal,2025-07-28 13:40:50
1,2,701-69-8742,Normal,2025-07-28 13:40:50
2,3,704-10-4056,Normal,2025-07-28 13:40:50
3,4,635-28-5728,Normal,2025-07-28 13:40:50
4,5,431-66-2305,Member,2025-07-28 13:40:50


In [10]:
table_name = "sales_predictions"
# Query entire table
sql_query = f"SELECT * FROM {table_name}"

# Load data into pandas DataFrame
df = pd.read_sql(sql_query, conn)
print(f"Loaded {len(df)} rows from '{table_name}' table.")

# Save to CSV
output_file = f"{table_name}.csv"
df.to_csv(output_file, index=False)
print(f"Table '{table_name}' successfully saved as '{output_file}'.")
   

Loaded 5 rows from 'sales_predictions' table.
Table 'sales_predictions' successfully saved as 'sales_predictions.csv'.


/tmp/ipykernel_339503/2701955691.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)
